In [ ]:
import os

from gnssrefl.utils import check_environment, set_environment, get_sys
from pathlib import Path

# Making sure environment variables are set - this is required to run the gnssrefl code
notebook_dir = Path.cwd().parents[0]

exists = check_environment()
if exists == False:
    #If you are running this locally - make sure the items in the exe folder have execution permissions
    set_environment(refl_code=str(notebook_dir),
                    orbits=str(notebook_dir / "orbits"),
                    exe=str(notebook_dir / "exe"))

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']
print("refl_code location:", refl_code_loc)

### Step 3. Download and check EXE dependencies are present:

use environment.download_crx2rnx to import the crx2rnx file(Required translator for compressed (Hatanaka) RINEX files) which is dependant on your working OS - this is required to run the gnssrefl code.

If this does not properly find your running os, then it will print out an error and instruct you how to add a parameter to manually set which os you are using.

Note that this function relies on your environment variables to be properly set.

In [ ]:
print('files in exe folder:', os.listdir(os.environ['EXE']))

#import gnssrefl rinex2snr function (environment variables must be set first)
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe

# Download the executables
try:
    os.environ['DOCKER']
except KeyError:
    sys = get_sys()
    installexe(sys)

If you see 'CRX2RNX' and 'gfzrnx' in your EXE folder then you are all set. 

**Note*** that the gfzrnx file was in the exe when you pulled the repository - it currently is set for a linux environment and can only be used with the docker version of the jupyter notebooks or if you are running linux. The gfzrnx file is not required to run the code - but is needed if you want to work with RINEX3 files. If you need to download the correct version for your os then download from [here](http://dx.doi.org/10.5880/GFZ.1.1.2016.002) and then place it in your exe folder.

### Step 4. Run a quick Analysis

#### a. simple use case that requires CRX2RNX and broadcast orbits:

In [ ]:
station = 'p042'
year = 2018
doy = 150

To understand what rinex2snr returns, lets look at the function's available and default parameters:

In [ ]:
rinex2snr?

Now lets run the function without changing any of the defaults.

In [ ]:
rinex2snr(station, year, doy)

you've successfully run the rinex2snr program that:
* downloaded and uncompressed [hatanaka](https://www.unavco.org/data/gps-gnss/hatanaka/hatanaka.html) rinex for a single station (p042) for a single day (doy 150 in 2018)
* downloaded GPS broadcast orbits
* calculated azimuth and elevation for each satellite at each epoch given these orbits
* wrote this az/el, signal, time and CN0 information to a formatted snr output file
for future analysis.
Reminder, the .66 file name suffix refers to the
[elevation masking options](https://github.com/kristinemlarson/gnssrefl#iv-rinex2snr---extracting-snr-data-from-rinex-files-).

#### b. simple use case that requires CRX2RNX and SP3 orbits:

Here we will run rinex2snr for the same day, but lets change the 'orb' parameter to gnss.

In [ ]:
rinex2snr(station, year, doy=doy, orb='gnss')

Note* If you get:
*SNR file exists...*\
This is because the logic of gnssrefl checks for an snr file prior for processing - and we already processed this day earlier.
Remember this fact if you ever want to **re**-process with different orbits!
You can use the overwrite parameter to overwrite files if you want to reprocess.
Now lets try that again.

In [ ]:
rinex2snr(station, year, doy=150, orb='gnss', overwrite=True)

If you get:
SUCCESS: SNR file was created: ...
you've successfully:

* downloaded and uncompressed hatanaka rinex for a single station (p042) for a single day (doy 150 in 2018)
* downloaded SP3 format GNSS orbits from the GFZ archive
* calculated azimuth and elevation for each satellite at each epoch
* wrote this az/el, signal, time and CN0 information to a formatted snr output file for future analysis.

#### c. (OPTIONAL - requires the gfzrnx executable mentioned previously ) RINEX 3 simple use case that requires gfzrnx

**If** you are interested in using RINEX version 3 data, please run this test:

note: this will fail if you do not have the correct system-dependant gfzrnx translation file. See the instructions above to get this file.

In [ ]:
rinex2snr(station='onsa00swe', year=2020, doy=1, archive='cddis', orb='gnss')

If you get:
*SUCCESS: SNR file was created: ...* \
you've successfully:
* downloaded and uncompressed rinex 3 for a single station (onsa)
for a single day (doy 1 in 2020) from the cddis archive
* converted rinex 3 to rinex 2 using gfzrnx executable
* downloaded SP3 format GNSS orbits from the GFZ archive
* calculated azimuth and elevation for each satellite at each epoch
* wrote this az/el, signal, time and CN0 information to a formatted
snr output file for future analysis.